In [1]:
import os

from dotenv import load_dotenv
from etg import GuestRoom

load_dotenv()

# ETG API Credentials
ETG_KEY_ID = os.environ["ETG_KEY_ID"]
ETG_API_KEY = os.environ["ETG_API_KEY"]
ETG_REQUEST_TIMEOUT = 30.0

SCORING_MODEL = os.environ["SCORING_MODEL"]

# Search Parameters
CITY = "Москва"

CHECKIN_DATE = "2026-03-02"
CHECKOUT_DATE = "2026-03-04"

CURRENCY = "RUB"
LANGUAGE = "ru"
RESIDENCY = "RU"

GUESTS: list[GuestRoom] = [{"adults": 2, "children": [4, 2]}]
LIMIT = 1000

# User preferences for AI
USER_PREFERENCES = "Обязательно две комнаты и две кровати. Хорошие отзывы. Чистота"

# Filters
MIN_PRICE: float | None = 3000.0  # None = no minimum
MAX_PRICE: float | None = 20000.0  # None = no maximum

# Save results to JSON
SAVE_RESULTS = True  # Set to False to disable JSON export

In [2]:
from etg import ETGClient
from utils import ostrovok_url

client = ETGClient(ETG_KEY_ID, ETG_API_KEY, timeout=ETG_REQUEST_TIMEOUT)

In [3]:
async def find_region_id(client: ETGClient, city_name: str, language: str) -> int | None:
    """Find region ID for a city name."""
    print(f"Looking up region ID for '{city_name}'...")
    regions = await client.suggest_region(city_name, language)

    if not regions:
        print(f"  No regions found for '{city_name}'")
        return None

    # Only accept City type
    for region in regions:
        if region["type"] == "City":
            region_id = region["id"]
            print(f"  Found: {region['name']} ({region.get('country_code', '')}), region_id={region_id}")
            return region_id

    # No city found - show available options
    print(f"  No city found. Available regions:")
    for region in regions[:5]:
        print(f"    - {region['name']} (type: {region['type']}, id: {region['id']})")

    return None

In [4]:
# Find region by city name.
# ETG API requires region_id for hotel search, so we first
# lookup the region ID via suggest_region by city name.
region_id = await find_region_id(client, CITY, LANGUAGE)
if not region_id:
    raise ValueError(f"Could not find region for '{CITY}'")

print(f"\nSearching hotels in {CITY}...")
print(f"  Dates: {CHECKIN_DATE} to {CHECKOUT_DATE}")
print(f"  Currency: {CURRENCY}, Limit: {LIMIT}")

  Found: Москва (RU), region_id=2395

Searching hotels in Москва...
  Dates: 2026-03-02 to 2026-03-04
  Currency: RUB, Limit: 1000


In [5]:
import pandas as pd

from services import filter_hotels_by_price

# Search available hotels in the region with given parameters.
# Filters by price range if MIN_PRICE/MAX_PRICE are set.
# Returns short hotel info: id, hid, and rates (room name, price, meal).
# Full content (name, address, amenities) is fetched separately via hotel content API.
# Search available hotels in the region with given parameters.
# Filters by price range if MIN_PRICE/MAX_PRICE are set.
# Returns short hotel info: id, hid, and rates (room name, price, meal).
# Full content (name, address, amenities) is fetched separately via hotel content API.
search_results = await client.search_hotels_by_region(
    region_id=region_id,
    checkin=CHECKIN_DATE,
    checkout=CHECKOUT_DATE,
    residency=RESIDENCY,
    guests=GUESTS,
    currency=CURRENCY,
    language=LANGUAGE,
    hotels_limit=LIMIT,
)

all_hotels = search_results.get("hotels", [])
total_available = search_results.get("total_hotels", len(all_hotels))

# Filter by price
hotels = filter_hotels_by_price(all_hotels, MIN_PRICE, MAX_PRICE)
total_after_filter = len(hotels)

In [6]:
from services import batch_get_content

hotel_ids = [h["hid"] for h in hotels]

print(f"[batch_get_content_start] Загрузка контента для {len(hotel_ids)} отелей...")
content_map = await batch_get_content(client, hotel_ids, LANGUAGE)
print(f"[batch_get_content_done] Загружен контент для {len(content_map)} из {len(hotel_ids)} отелей")

[batch_get_content_done] Загружен контент для 385 из 385 отелей


In [7]:
from services import batch_get_reviews, filter_reviews

print(f"[batch_get_reviews_start] Загрузка отзывов для {len(hotel_ids)} отелей...")
raw_reviews_payload = await batch_get_reviews(client, hotel_ids, LANGUAGE)
reviews_map = filter_reviews(raw_reviews_payload)

total_raw = sum(rd["total_reviews"] for rd in raw_reviews_payload.values())
total_filtered = sum(len(rd["reviews"]) for rd in reviews_map.values())
# Compute filtered_by_age: how many reviews were filtered out
total_raw_in_filtered = sum(rd["total_reviews"] for rd in reviews_map.values())
total_filtered_by_age = total_raw_in_filtered - total_filtered
hotels_with_reviews = len(reviews_map)

# Calculate average rating across all hotels
all_avg_ratings = [rd["avg_rating"] for rd in reviews_map.values() if rd["avg_rating"] is not None]
overall_avg = sum(all_avg_ratings) / len(all_avg_ratings) if all_avg_ratings else 0

print(
    f"[batch_get_reviews_done] Всего {hotels_with_reviews} отелей с отзывами из {len(hotel_ids)}"
)
print(
    f"  Обработано {total_raw} отзывов → {total_filtered} релевантных "
    f"(отсечено по давности: {total_filtered_by_age})"
)
print(f"  Средний рейтинг: {overall_avg:.1f}/10")

[batch_get_reviews_done] Всего 380 отелей с отзывами из 385
  Обработано 54329 отзывов → 43507 релевантных (отсечено по давности: 10822)
  Средний рейтинг: 8.5/10


In [8]:
from services import combine_hotels_data

combined = combine_hotels_data(hotels, content_map, reviews_map)
print(f"Combined {len(combined)} hotels with content and reviews")

Combined 385 hotels with content and reviews


In [9]:
import json
from pathlib import Path

if SAVE_RESULTS:
    # Create .artifacts directory if it doesn't exist
    artifacts_dir = Path(".artifacts")
    artifacts_dir.mkdir(exist_ok=True)
    
    # Format filename: City_CheckinDate_CheckoutDate.json
    filename = f"{CITY}_{CHECKIN_DATE}_{CHECKOUT_DATE}.json"
    filepath = artifacts_dir / filename
    
    # Prepare export data with search metadata at top level
    export_data = {
        "search": {
            "city": CITY,
            "region_id": region_id,
            "checkin": CHECKIN_DATE,
            "checkout": CHECKOUT_DATE,
            "guests": GUESTS,
            "min_price": MIN_PRICE,
            "max_price": MAX_PRICE,
            "currency": CURRENCY,
            "language": LANGUAGE,
            "residency": RESIDENCY,
        },
        "hotels": combined,  # HotelFull[] - already has all data
        "stats": {
            "total_hotels": len(combined),
            "total_available": total_available,
            "total_after_filter": len(hotels),
        }
    }
    
    # Save to JSON
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(export_data, f, ensure_ascii=False, indent=2)
    
    # Format guests for display
    guests_str = "; ".join([
        f"{g['adults']} adults" + (f" + {len(g.get('children', []))} children (ages: {', '.join(map(str, g.get('children', [])))})" if g.get('children') else "")
        for g in GUESTS
    ])
    
    print(f"✅ Saved {len(combined)} hotels to {filepath}")
    print(f"   Search: {CITY}, {CHECKIN_DATE} → {CHECKOUT_DATE}")
    print(f"   Guests: {guests_str}")
    print(f"   Price: {MIN_PRICE} - {MAX_PRICE} {CURRENCY}")
else:
    print("⏭️  JSON export disabled (SAVE_RESULTS = False)")

✅ Saved 385 hotels to .artifacts/Москва_2026-03-02_2026-03-04.json
   Search: Москва, 2026-03-02 → 2026-03-04
   Guests: 2 adults + 2 children (ages: 4, 2)
   Price: 3000.0 - 20000.0 RUB


In [10]:
import json

from services import estimate_tokens, prepare_hotel_for_llm, presort_hotels

# Parameters for review sampling
MAX_REVIEWS_PER_HOTEL = 30
REVIEW_TEXT_MAX_LENGTH = 512

# Estimate tokens before presort
hotels_for_llm_all = [
    prepare_hotel_for_llm(h, MIN_PRICE, MAX_PRICE, MAX_REVIEWS_PER_HOTEL, REVIEW_TEXT_MAX_LENGTH)
    for h in combined
]
tokens_before = estimate_tokens(json.dumps(hotels_for_llm_all, ensure_ascii=False), SCORING_MODEL)

# Pre-sort by hotel kind tier and prescore, limit to top 100 for LLM scoring
PRESORT_LIMIT = 120
top_hotels = presort_hotels(combined, reviews_map, limit=PRESORT_LIMIT)

# Estimate tokens after presort
hotels_for_llm_top = [
    prepare_hotel_for_llm(h, MIN_PRICE, MAX_PRICE, MAX_REVIEWS_PER_HOTEL, REVIEW_TEXT_MAX_LENGTH)
    for h in top_hotels
]
tokens_after = estimate_tokens(json.dumps(hotels_for_llm_top, ensure_ascii=False), SCORING_MODEL)

print(f"[presort_done] {len(combined)} отелей → {len(top_hotels)} (лимит {PRESORT_LIMIT})")
print(f"  Токены: ~{tokens_before:,} → ~{tokens_after:,} (экономия {tokens_before - tokens_after:,})")

[presort_done] 385 отелей → 120 (лимит 120)
  Токены: ~969,726 → ~377,961 (экономия 591,765)


In [11]:
import time

from services import finalize_scored_hotels, score_hotels

# Score hotels using single LLM request
print(f"[scoring_start] Scoring {len(top_hotels)} hotels...")
start_time = time.time()

scoring_result = await score_hotels(
    top_hotels,
    USER_PREFERENCES,
    guests=GUESTS,
    max_reviews=MAX_REVIEWS_PER_HOTEL,
    review_text_max_length=REVIEW_TEXT_MAX_LENGTH,
    min_price=MIN_PRICE,
    max_price=MAX_PRICE,
    currency=CURRENCY,
    top_count=10
)

elapsed = time.time() - start_time

if scoring_result["error"]:
    print(f"\n❌ ERROR: {scoring_result['error']}")
    scored_hotels = None
else:
    scoring_results = scoring_result["results"]
    print(f"[scoring_done] {len(scoring_results)} hotels scored — {elapsed:.1f}s")
    print(f"  Estimated tokens: ~{scoring_result['estimated_tokens']:,}")
    
    # Finalize scored hotels - merge scoring results with full hotel data
    scored_hotels = finalize_scored_hotels(combined, scoring_results)
    print(f"\n[finalize_done] {len(scored_hotels)} hotels with complete data")

[scoring_done] 10 hotels scored — 45.6s
  Estimated tokens: ~381,559

[finalize_done] 10 hotels with complete data


In [12]:
from services import HotelScored


def _extract_hotel_display_data(
    hotel: HotelScored,
) -> dict:
    """Extract display fields from a scored hotel."""
    hotel_id = hotel["id"]
    name = hotel["name"]
    hid = hotel["hid"]
    kind = hotel.get("kind", "")
    score = hotel["score"]
    reasons = hotel.get("top_reasons", [])
    penalties = hotel.get("score_penalties", [])
    selected_hash = hotel.get("selected_rate_hash")

    # Find selected rate by hash
    rates = hotel.get("rates", [])
    selected_rate = next((r for r in rates if r.get("match_hash") == selected_hash), None)

    # Get rate details
    if selected_rate:
        room_name = selected_rate.get("room_name", "")[:50]
        meal_data = selected_rate.get("meal_data", {})
        meal = meal_data.get("value", selected_rate.get("meal", ""))

        daily_prices = selected_rate.get("daily_prices", [])
        if daily_prices:
            total_price = sum(float(p) for p in daily_prices)
            num_nights = len(daily_prices)
            avg_price_per_night = total_price / num_nights if num_nights > 0 else 0
            pt = selected_rate.get("payment_options", {}).get("payment_types", [])
            currency = pt[0].get("show_currency_code", "") if pt else ""
            total_price_str = f"{total_price:.0f} {currency}"
            avg_price_str = f"{avg_price_per_night:.0f} {currency}"
        else:
            pt = selected_rate.get("payment_options", {}).get("payment_types", [])
            if pt:
                total_price = float(pt[0].get("show_amount", 0))
                currency = pt[0].get("show_currency_code", "")
                total_price_str = f"{total_price:.0f} {currency}"
                avg_price_str = f"{total_price:.0f} {currency}"
            else:
                total_price_str = "N/A"
                avg_price_str = "N/A"
    else:
        room_name = "N/A"
        meal = "N/A"
        total_price_str = "N/A"
        avg_price_str = "N/A"

    reviews = hotel.get("reviews")
    avg_rating = reviews.get("avg_rating") if reviews else None
    detailed = reviews.get("detailed_averages", {}) if reviews else {}

    url = ostrovok_url(
        hotel_id=hotel_id,
        hid=hid,
        checkin=CHECKIN_DATE,
        checkout=CHECKOUT_DATE,
        guests=GUESTS,
        region_id=region_id,
    )

    return {
        "hotel_id": hotel_id,
        "name": name,
        "hid": hid,
        "kind": kind,
        "score": score,
        "reasons": reasons,
        "penalties": penalties,
        "room_name": room_name,
        "meal": meal,
        "total_price_str": total_price_str,
        "avg_price_str": avg_price_str,
        "avg_rating": avg_rating,
        "detailed": detailed,
        "url": url,
    }


def print_top_hotels(
    scored_hotels: list[HotelScored],
    top_n: int = 10,
) -> None:
    """Print top N scored hotels with details and Ostrovok links."""
    print(f"\n{'='*80}")
    print(f"TOP {top_n} HOTELS")
    print(f"{'='*80}\n")

    for i, hotel in enumerate(scored_hotels[:top_n], 1):
        d = _extract_hotel_display_data(hotel)
        print(f"{i}. {d['name']} [{d['kind']}]")
        if d["avg_rating"]:
            print(f"   Score: {d['score']}/100 | Rating: {d['avg_rating']}/10")
        else:
            print(f"   Score: {d['score']}/100")
        print(f"   Room: {d['room_name']}")
        print(f"   Total: {d['total_price_str']} | Avg per night: {d['avg_price_str']} | Meal: {d['meal']}")
        if d["reasons"]:
            print(f"   + {'; '.join(d['reasons'][:3])}")
        if d["penalties"]:
            print(f"   - {'; '.join(d['penalties'][:5])}")
        print(f"   🔗 {d['url']}")
        print()

    selected = min(top_n, len(scored_hotels))
    print(f"Всего найдено {len(combined)} отелей на эти даты.")
    print(f"Подобраны лучшие {selected} по вашим критериям.")


print_top_hotels(scored_hotels, top_n=10)


TOP 10 HOTELS

1. Барин Резиденс Центр [Hotel]
   Score: 95/100 | Rating: 9.5/10
   Room: Апартаменты Двухкомнатные (питание для детей не вк
   Total: 34380 RUB | Avg per night: 17190 RUB | Meal: nomeal
   + Excellent 9.5 rating with 9.4 cleanness score; Two-bedroom apartment perfectly matches the 2-room requirement; Prime location with a 9.9 score near the Kremlin
   - avg_rating 9.5 - below 9.5 threshold for perfect score; price positioning at 83% of range
   🔗 https://ostrovok.ru/hotel/russia/moscow/mid8854270/barin_rezidence_balchug/?dates=02.03.2026-04.03.2026&guests=2and4.2&q=2395

2. Городской отель Дом Купца Бавыкина [Hotel]
   Score: 93/100 | Rating: 9.6/10
   Room: Четырёхместный номер с проекционным кинотеатром се
   Total: 19034 RUB | Avg per night: 9517 RUB | Meal: nomeal
   + Exceptional 9.8 cleanness score and 9.6 overall rating; Explicit two-room family suite with projection cinema; Great value for money at the lower end of the budget range
   - 3-star property baselin

In [13]:
def top_hotels_dataframe(
    scored_hotels: list[HotelScored],
    top_n: int = 10,
) -> pd.DataFrame:
    """Build a DataFrame with top N scored hotels."""
    data = []
    for hotel in scored_hotels[:top_n]:
        d = _extract_hotel_display_data(hotel)
        data.append({
            "name": d["name"][:35],
            "kind": d["kind"],
            "room": d["room_name"][:30],
            "total": d["total_price_str"],
            "avg/night": d["avg_price_str"],
            "meal": d["meal"],
            "score": d["score"],
            "rating": d["avg_rating"],
            "clean": d["detailed"].get("cleanness"),
            "url": d["url"],
        })

    df = pd.DataFrame(data)
    df.index = range(1, len(df) + 1)
    return df


pd.set_option("display.max_colwidth", 100)
top_hotels_dataframe(scored_hotels, top_n=10)

,name,kind,room,total,avg/night,meal,score,rating,clean,url
1,Барин Резиденс Центр,Hotel,Апартаменты Двухкомнатные (пит,34380 RUB,17190 RUB,nomeal,95,9.5,9.4,https://ostrovok.ru/hotel/russia/moscow/mid8854270/barin_rezidence_balchug/?dates=02.03.2026-04....
2,Городской отель Дом Купца Бавыкина,Hotel,Четырёхместный номер с проекци,19034 RUB,9517 RUB,nomeal,93,9.6,9.8,https://ostrovok.ru/hotel/russia/moscow/mid11347687/dom_kuptsa_bavykina_mini_hotel/?dates=02.03....
3,Отель Звёзды Арбата (ранее Мариотт,Hotel,Двухместный номер Делюкс (2 от,32336 RUB,16168 RUB,nomeal,91,9.2,9.2,https://ostrovok.ru/hotel/russia/moscow/mid8139930/moscow_marriott_hotel_novy_arbat/?dates=02.03...
4,Select Hotel Paveletskaya,Hotel,Двухместный люкс с 2 комнатами,25800 RUB,12900 RUB,nomeal,90,9.1,9.2,https://ostrovok.ru/hotel/russia/moscow/mid7596982/tatiana/?dates=02.03.2026-04.03.2026&guests=2...
5,Отель Севастополь Гранд Классик,Hotel,Четырёхместные апартаменты (пи,27150 RUB,13575 RUB,nomeal,89,9.2,9.3,https://ostrovok.ru/hotel/russia/moscow/mid7625812/sevastopol_hotel_bld2/?dates=02.03.2026-04.03...
6,Отель Никоновка,Hotel,Номер Супериор комфорт с 2 ком,17720 RUB,8860 RUB,breakfast,87,9.0,9.1,https://ostrovok.ru/hotel/russia/moscow/mid6730849/gostinitsa_nikonovka/?dates=02.03.2026-04.03....
7,Отель Апельсин Чистые Пруды,Hotel,Люкс семейный с 2 комнатами,21150 RUB,10575 RUB,breakfast,86,9.0,9.1,https://ostrovok.ru/hotel/russia/moscow/mid8868870/otel_apelsin_chistye_prudy/?dates=02.03.2026-...
8,Отель Лесная Сафмар (бывший Холидей,Hotel,Двухместный люкс с большой дву,36480 RUB,18240 RUB,breakfast,84,9.0,9.1,https://ostrovok.ru/hotel/russia/moscow/mid7467380/kholidei_inn_moskva_lesnaia/?dates=02.03.2026...
9,Бутик-Отель Регул,Boutique_and_Design,Четырёхместный номер Семейный,36587 RUB,18294 RUB,nomeal,82,8.9,9.0,https://ostrovok.ru/hotel/russia/moscow/mid8848163/russkie_sezonyi/?dates=02.03.2026-04.03.2026&...
10,Отель Mamaison All-Suites Spa Pokro,Hotel,Suite Deluxe 1 Bedroom (питани,30000 RUB,15000 RUB,nomeal,81,8.8,8.9,https://ostrovok.ru/hotel/russia/moscow/mid7590428/mamaison_allsuites_spa_hotel_pokrovka/?dates=...
